<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/An%C3%A1lisis_Temporal_Anomal%C3%ADas_Aus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PUNTOS_FIJOS_MD (CSV)


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,04/10/2021,Derecha,PF01 PRIMA,0.000333,-0.000667,0.003333,0.000745,0.745356,296.565051,0.041667,-0.083333,0.416667,0.093169
1,10/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.004333,0.000943,0.942809,225.000000,-0.047619,-0.047619,0.309524,0.067344
2,14/10/2021,Derecha,PF01 PRIMA,0.003333,0.003333,0.003333,0.004714,4.714045,45.000000,0.185185,0.185185,0.185185,0.261891
3,18/10/2021,Derecha,PF01 PRIMA,0.002333,0.002333,0.000333,0.003300,3.299832,45.000000,0.106061,0.106061,0.015152,0.149992
4,27/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.005333,0.000943,0.942809,225.000000,-0.021505,-0.021505,0.172043,0.030413


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# ========= Selectores generales =========
instrumento_dropdown = widgets.Dropdown(
    options=[
        "Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
        "Celdas de Asentamiento", "Freatímetros", "Extensómetros"
    ],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

# ========= Funciones de actualización de selectores =========
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    if 'FECHA' not in df.columns:
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique().tolist())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'FECHA' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique().tolist())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_inc(change=None):
    df = datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]
    if df.empty or 'Inclinometro' not in df.columns or 'Fecha' not in df.columns:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = ["Todos"]
        eje_dropdown.options = ["A+", "A-", "B+", "B-"]
        return
    df = df.copy()
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = ["Todos"] + sorted(df['Inclinometro'].dropna().unique().tolist())
    anio_inc_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['Fecha'].dt.year.dropna().unique())]
    eje_dropdown.options = ["A+", "A-", "B+", "B-"]

def actualizar_opciones_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_ca_dropdown.options:
        progresiva_ca_dropdown.value = progresiva_ca_dropdown.options[0]
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    celdas = sorted(df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]['CELDA_DE_ASENTAMIENTO'].dropna().unique().tolist())
    celda_dropdown.options = ["Todas"] + list(celdas)

def actualizar_opciones_fr(change=None):
    df = datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]
    if df.empty or 'FREATIMETRO' not in df.columns or 'FECHA' not in df.columns:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique().tolist())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_ex(change=None):
    df = datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]
    if df.empty or 'EXTENSOMETRO' not in df.columns or 'FECHA' not in df.columns:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique().tolist())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# ========= Selectores de anomalía =========
metodo_anomalia = widgets.Dropdown(
    options=[
        ("Tukey (IQR)", "tukey"),
        ("Z-score", "zscore"),
        ("Umbral manual", "manual")
    ],
    value="tukey",
    description="Método:"
)
umbral_manual = widgets.FloatText(
    value=0.0,
    description="Umbral (> valor):",
    disabled=True
)

def actualizar_umbral_manual(change):
    umbral_manual.disabled = (metodo_anomalia.value != "manual")
metodo_anomalia.observe(actualizar_umbral_manual, names="value")

# ========= Botón y output =========
boton_temporal = widgets.Button(
    description="Graficar serie temporal y anomalías",
    button_style="info"
)
output_temporal = widgets.Output()

def analizar_temporal_anomalias(b=None):
    with output_temporal:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        variable = None
        df = None
        fecha_col = 'FECHA'
        titulo = ""

        # --- Filtrado según instrumento (idéntico a los otros análisis) ---
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)].copy()
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value].copy()
            fecha_col = "FECHA"
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            variable = variable_pe_dropdown.value
            df = df.copy()
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value].copy()
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value].copy()
            if anio_pe_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)].copy()
            fecha_col = "FECHA"
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            variable = variable_cg_dropdown.value
            df = df.copy()
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value].copy()
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value].copy()
            if anio_cg_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)].copy()
            fecha_col = "FECHA"
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            fecha_col = 'Fecha'
            variable = eje_dropdown.value
            df = df.copy()
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value].copy()
            if anio_inc_dropdown.value != "Todos":
                df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)].copy()
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            variable = variable_ca_dropdown.value
            df = df.copy()
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value].copy()
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value].copy()
            if anio_ca_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)].copy()
            fecha_col = "FECHA"
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            variable = variable_fr_dropdown.value
            df = df.copy()
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value].copy()
            if anio_fr_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)].copy()
            fecha_col = "FECHA"
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            variable = variable_ex_dropdown.value
            df = df.copy()
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value].copy()
            if anio_ex_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)].copy()
            fecha_col = "FECHA"
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido."); return

        # Validación básica
        if variable not in df.columns or fecha_col not in df.columns:
            print(f"⚠️ Variable o fecha no disponible en los datos."); return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[fecha_col, variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar."); return

        # Orden y limpieza
        df = df.sort_values(fecha_col)
        fechas = df[fecha_col]
        valores = df[variable]

        # ====== Detección de anomalías ======
        metodo = metodo_anomalia.value
        if metodo == "tukey":
            q1 = valores.quantile(0.25)
            q3 = valores.quantile(0.75)
            iqr = q3 - q1
            lim_inf = q1 - 1.5 * iqr
            lim_sup = q3 + 1.5 * iqr
            mask_anomalia = (valores < lim_inf) | (valores > lim_sup)
            metodo_str = f"Tukey: [{lim_inf:.2f}, {lim_sup:.2f}]"
        elif metodo == "zscore":
            media = valores.mean()
            std = valores.std()
            z_scores = (valores - media) / std
            mask_anomalia = (np.abs(z_scores) > 3)
            metodo_str = f"Z-score > 3 (media={media:.2f}, std={std:.2f})"
        elif metodo == "manual":
            umbral = umbral_manual.value
            mask_anomalia = (valores > umbral)
            metodo_str = f"Mayor que {umbral:.2f}"
        else:
            print("Método de anomalía no reconocido."); return

        # ====== Gráfico ======
        plt.figure(figsize=(12,5))
        plt.plot(fechas, valores, label="Serie", color="#1866a3", marker='o', markersize=4)
        plt.scatter(fechas[mask_anomalia], valores[mask_anomalia], color="red", label="Anomalías", zorder=2)
        plt.xlabel("Fecha")
        plt.ylabel(variable)
        plt.title(f"Serie temporal de {variable} ({titulo})\nAnomalías detectadas: {metodo_str}")
        plt.legend()
        plt.tight_layout()
        plt.show()

        # ====== Resumen ======
        n_anom = mask_anomalia.sum()
        print(f"Se detectaron {n_anom} anomalías de {len(valores)} datos ({100*n_anom/len(valores):.1f}%) usando: {metodo_str}")
        if n_anom > 0:
            display(df.loc[mask_anomalia, [fecha_col, variable]].reset_index(drop=True).head(10))

# ========= Conexiones y display =========
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_temporal.on_click(analizar_temporal_anomalias)

display(HTML("<h3 style='color:#1866a3'>Análisis temporal de anomalías</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown, metodo_anomalia, umbral_manual]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))
display(boton_temporal)
display(output_temporal)
actualizar_controles_visibles()

Button(button_style='info', description='Graficar serie temporal y anomalías', style=ButtonStyle())

Output()